In [1]:
!pip install sacremoses

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 17.5 MB/s eta 0:00:00a 0:00:01


In [2]:
# Directory paths
bootstrapped_level_1_path = '/kaggle/input/botstrapped-level-2-gpt/BootStrapped-Level-2-GPT/BootStrapped-Level-1-filter-Gpt'
summarized_abstracts_path = '/kaggle/input/botstrapped-level-2-gpt/BootStrapped-Level-2-GPT/summarized_abstracts_gpt'
patient_histories_path = '/kaggle/input/botstrapped-level-2-gpt/BootStrapped-Level-2-GPT/Patient_Histories_Cleaned.csv'

In [3]:
# Function to extract patient number from file name
def extract_patient_number(filename):
    # Extracting the patient number from filenames like 'summarized_patient-numberX-articles.csv'
    # or 'Patient-X-results-with-links.csv'
    parts = filename.split('-')
    for part in parts:
        if part.isdigit():
            return part
    return None

# Load the patient histories data
patient_histories_df = pd.read_csv(patient_histories_path)

In [4]:
from transformers import BioGptForSequenceClassification, BioGptTokenizer

model_name = "stanford-crfm/BioMedLM"
model = BioGptForSequenceClassification.from_pretrained(model_name, num_labels=3, problem_type="multi_label_classification")
tokenizer = BioGptTokenizer.from_pretrained(model_name)

from transformers import pipeline

classifier = pipeline("sentiment-analysis", model=model_name, tokenizer=model_name)

You are using a model of type gpt2 to instantiate a model of type biogpt. This is not supported for all configurations of models and can yield errors.


Some weights of BioGptForSequenceClassification were not initialized from the model checkpoint at stanford-crfm/BioMedLM and are newly initialized: ['layers.4.self_attn.out_proj.weight', 'layers.6.self_attn.v_proj.bias', 'layers.9.fc2.weight', 'layers.7.self_attn.q_proj.bias', 'layers.20.final_layer_norm.bias', 'layers.19.self_attn.v_proj.weight', 'layers.15.self_attn.q_proj.weight', 'layers.20.self_attn_layer_norm.bias', 'layers.18.self_attn.out_proj.bias', 'layers.21.fc1.weight', 'layers.9.self_attn.v_proj.bias', 'layers.16.self_attn.q_proj.bias', 'layers.3.fc2.bias', 'layers.15.final_layer_norm.weight', 'layers.8.fc1.bias', 'layers.18.final_layer_norm.weight', 'layers.20.self_attn_layer_norm.weight', 'layers.1.self_attn.k_proj.bias', 'layers.20.fc2.bias', 'layers.0.self_attn.q_proj.weight', 'layers.6.self_attn.v_proj.weight', 'layers.20.self_attn.v_proj.bias', 'layers.19.fc1.weight', 'layers.14.self_attn.out_proj.weight', 'layers.21.self_attn.out_proj.weight', 'layers.3.self_attn.ou

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'BioGptTokenizer'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at stanford-crfm/BioMedLM and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
def get_decision(question):
    inputs = tokenizer(question, return_tensors="pt", max_length=512, truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)
    
    predicted_class_idx = outputs.logits.argmax(dim=-1).item()
    
    return predicted_class_idx

MAX_TOKENS = 512

def truncate_text_to_fit(text, max_tokens=MAX_TOKENS):
    token_count = len(tokenizer.tokenize(text))
    if token_count > max_tokens - 2:  
        truncated_text = " ".join(text.split()[:-1])
        while len(tokenizer.tokenize(truncated_text)) > max_tokens - 2:
            truncated_text = " ".join(truncated_text.split()[:-1])
    else:
        truncated_text = text
        
    return truncated_text

def create_input_text(context, question):
    input_text =  "is it a relevant diagnosis: "+ question +  " in this context: " + context
    return input_text

def create_df(patient_id,indexes, pubids, results):
    df = pd.DataFrame({
        'Index': indexes,
        'PubID': pubids,
        'Result': results
    })
    return df
    #df.to_csv(str(patient_id)+'-results.csv', index=False)
    
label_mapping = {
    "LABEL_0": "yes",
    "LABEL_1": "no",
    "LABEL_2": "maybe"
}

def map_label(results):
    try:
        # Initialize an empty list to hold the mapped labels
        mapped_labels = []
        # Iterate over each result in the list
        for result in results:
            # Assuming each result is a list with a single dictionary,
            # extract the label from that dictionary
            label = result['label']
            # Map the label and append to the list of mapped labels
            mapped_labels.append(label_mapping.get(label, "Unknown"))
        return mapped_labels[0]

    except Exception as e:
        # Return the error message for debugging
        return f"Error in processing: {e}"

In [6]:
# Loop to process and handle each patient's data individually
for file in os.listdir(bootstrapped_level_1_path):
    if file.endswith('.csv'):
        patient_number = extract_patient_number(file)
        if patient_number is None:
            continue

        # Load and filter the result file
        result_df = pd.read_csv(os.path.join(bootstrapped_level_1_path, file))
        filtered_results = result_df[result_df['Result'].isin(['yes', 'maybe'])]

        # Load and filter the abstracts file for this patient
        abstracts_filename = f"summarized_patient-number{patient_number}-articles.csv"
        abstracts_path = os.path.join(summarized_abstracts_path, abstracts_filename)
        if os.path.exists(abstracts_path):
            abstracts_df = pd.read_csv(abstracts_path)
            filtered_abstracts = abstracts_df[abstracts_df['ID'].isin(filtered_results['PubID'])]
        else:
            filtered_abstracts = pd.DataFrame()

        # Get the diagnosis for this patient from the patient history file
        patient_diagnosis = patient_histories_df[patient_histories_df['Patient ID'] == f"Patient-{patient_number}"]['Diagnosis'].iloc[0]
        # Combine the results and abstracts into one DataFrame and add the diagnosis as a new column
        combined_df = pd.merge(filtered_results, filtered_abstracts, left_on='PubID', right_on='ID', how='left')
        combined_df['Diagnosis'] = patient_diagnosis
        combined_df['input_text'] = combined_df.apply(lambda row: create_input_text(row['Summary'], row['Diagnosis']), axis=1)
        results = []
        pubids = []
        indexes = []
        for idx, row in combined_df.iterrows():
            indexes.append(idx + 1)
            pubids.append(row['PubID'])
            results.append(classifier(truncate_text_to_fit(row['input_text'])))

        if results:
            result=create_df(patient_number,indexes, pubids, results)
            result['Result'] = result['Result'].apply(map_label)
            new_file_name = f"/kaggle/working/Patient-{patient_number}-final-results-gpt.csv"
            result.to_csv(new_file_name, index=False)
            print(f"Processed, and saved: {new_file_name}")

Processed, and saved: /kaggle/working/Patient-3-final-results-gpt.csv
Processed, and saved: /kaggle/working/Patient-7-final-results-gpt.csv
Processed, and saved: /kaggle/working/Patient-4-final-results-gpt.csv
Processed, and saved: /kaggle/working/Patient-6-final-results-gpt.csv
Processed, and saved: /kaggle/working/Patient-5-final-results-gpt.csv
Processed, and saved: /kaggle/working/Patient-12-final-results-gpt.csv
Processed, and saved: /kaggle/working/Patient-11-final-results-gpt.csv
Processed, and saved: /kaggle/working/Patient-9-final-results-gpt.csv
Processed, and saved: /kaggle/working/Patient-13-final-results-gpt.csv
Processed, and saved: /kaggle/working/Patient-10-final-results-gpt.csv
Processed, and saved: /kaggle/working/Patient-14-final-results-gpt.csv
Processed, and saved: /kaggle/working/Patient-15-final-results-gpt.csv
Processed, and saved: /kaggle/working/Patient-8-final-results-gpt.csv
Processed, and saved: /kaggle/working/Patient-16-final-results-gpt.csv
Processed, an